<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/draft_few_shot_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
import numpy
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, pipeline

# test document
text = 'Daniel uses SimCSE to train a legal-oriented Longformer model.'

# Longformer as classifier
tokenizer = AutoTokenizer.from_pretrained('danielsaggau/longformer_simcse_scotus', 
                                          use_auth_token='XXXX',)
model = AutoModelForSequenceClassification.from_pretrained('danielsaggau/longformer_simcse_scotus', 
                                                           use_auth_token='XXXX',
                                                           num_labels=10)

torch_inputs = tokenizer([text], padding='max_length', max_length=1024, truncation=True, return_tensors='pt')

outputs = model(torch_inputs['input_ids'], attention_mask=torch_inputs['attention_mask'])

print(outputs[0].shape)

# LongformerModel as feature extractor
model_featurizer = AutoModel.from_pretrained('danielsaggau/longformer_simcse_scotus',
                                             use_auth_token='XXXX')

features = model_featurizer(torch_inputs['input_ids'], attention_mask=torch_inputs['attention_mask'])

# cls pooling
cls_pooled_doc_embedding = features[0][:, 0]
print(cls_pooled_doc_embedding.shape)

# mean pooling (default in SentenceTransformers)
mean_pooled_doc_embedding = features[0].mean(dim=1)
print(mean_pooled_doc_embedding.shape)

# max pooling
max_pooled_doc_embedding = features[0].max(dim=1)[0]
print(max_pooled_doc_embedding.shape)

# Longformer as feature extractor via pipeline
featurizer = pipeline('feature-extraction', 
                      'danielsaggau/longformer_simcse_scotus', 
                      use_auth_token='XXXX')

features = featurizer(text)

# cls pooling
cls_pooled_doc_embedding = features[0][0]

# mean pooling (default in SentenceTransformers)
mean_pooled_doc_embedding = numpy.mean(features[0], axis=0)
print(mean_pooled_doc_embedding.shape)

# max pooling
max_pooled_doc_embedding = numpy.max(features[0], axis=0)
print(max_pooled_doc_embedding.shape)

Downloading:   0%|          | 0.00/167M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at danielsaggau/longformer_simcse_scotus and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([1, 10])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
(512,)
(512,)


In [ ]:
from datasets import load_dataset
dataset = load_dataset("lex_glue", "scotus")

In [11]:
from transformers import set_seed
set_seed(123)

In [ ]:

# get doc features given automodel or pipelne
test_features[k] = [F1, …, F512]

label_1_repr = [F1, …, F512]  # a random training sample labeled with 1 (same for all Longformers, LegalLongformer, LegalLongformer+SimCSE, LegalLongformer+SimCSE+Bregman) need to fix the seed to do that ;) 
label_2_repr = [F1, …, F512] # a random training sample labeled with 2
…
label_14_repr = [F1, …, F512] # a random training sample labeled with 14


# compute cosine similarities
cos_sim_label_1 = cosine_sim(test_features[k], label_1_repr)
cos_sim_label_2 = cosine_sim(test_features[k], label_1_repr)
…
cos_sim_label_14 = cosine_sim(test_features[k], label_1_repr)

# find most likely label
test_label = argmax(cos_sim_label_1, cos_sim_label_2, …, cos_sim_label_14)

